# Trello Summary Notebook (Work In Progress)

This notebook scans a user's Trello boards to create a summary of your recent and upcoming work. It pulls out lists with specific names and places their boards in a summary document. I cut-and-paste the content of these documents into my weekly [snippets](http://blog.idonethis.com/google-snippets-internal-tool/). 

If you place an estimate of the time needed to complete a task in brackets preceeding the title of your cards, it will also calculate how much work you have in all lists named *To Do* and compare this against the available hours in your work week. Consequently, the assumption is that you're running this once a week.

Additionally, it creates a time tracking document with the titles of cards from all lists named *To Do*. This document has timers for each task and works as a punch clock, noting when you are working and what you are working on. 

*Note: your time estimates are not included in your summary document. These are for you and you alone. Likewise, I feel that the content of your time tracking document should be for your use only. This is about giving you tools to improve your efficency, not enableing some Orwellian tracking system.*

## Workflow

Set up a Trello borad for each of your projects, and create the following lists:

- **Backlog.** Things you want to get to someday.
- **On Deck.** Things you want to get to once *Doing* has room.
- **Waiting.** Things you are waiting on. *Eventually, I would like to eliminate this and rely on assigned team members.*
- **Doing.** Things you plan to do in the upcoming week.
- **Done.** Things you have done. 

Define bite-sized tasks for your projects and add these as cards to the appropriate lists, prefacing each task with an estimate in hours of how long you think it will take. 

Set up [IFTTT](https://ifttt.com/) recipes to add recurring tasks to your boards. For example, every week late Sunday night, IFTTT adds a card to my *Housekeeping & Misc* board for my weekly meetings. This garentees that when I run my script, this meeting time is accounted for in my available work hours. 

Once a week, review and edit your boards. Then run the code found under ***Run Me*** to produce a summary and time traking document.


## One-Time Setup

If you've already set things up, skip to *Run Me* below. 

### Dependancies

You'll need to install the [py-trello libary](http://py-trello.readthedocs.io/en/stable/). So run the following on the command line:

`pip install py-trello`

### Credentials

Login to your [Trello account](https://trello.com/) and vist [this page](https://trello.com/app-key) to get your app credentials (i.e., your *key* and your *secret*). When you run the cell below, you will be prompted to provide these. Now run the following code:

In [ ]:
from trello import TrelloClient
import os

try:
    inputFunc = raw_input
except NameError:
    inputFunc = input
    
os.environ['TRELLO_API_KEY'] = inputFunc('What is your TRELLO_API_KEY? ')
os.environ['TRELLO_API_SECRET'] = inputFunc('What is your TRELLO_API_SECRET? ')

credetials = open("keys.txt", "w")
credetials.write("%s,%s" %(os.environ['TRELLO_API_KEY'],os.environ['TRELLO_API_SECRET']))
credetials.close()

os.environ['TRELLO_EXPIRATION'] = 'never'

print ("\nYou keys have benn saved.\n")

# h/t @sarumont et al. See https://github.com/sarumont/py-trello/blob/master/trello/util.py

from __future__ import with_statement, print_function, absolute_import
from requests_oauthlib import OAuth1Session

def create_oauth_token(expiration=None, scope=None, key=None, secret=None, name=None, output=True):
    """
    Script to obtain an OAuth token from Trello.

    Must have TRELLO_API_KEY and TRELLO_API_SECRET set in your environment
    To set the token's expiration, set TRELLO_EXPIRATION as a string in your
    environment settings (eg. 'never'), otherwise it will default to 30 days.

    More info on token scope here:
        https://trello.com/docs/gettingstarted/#getting-a-token-from-a-user
    """
    request_token_url = 'https://trello.com/1/OAuthGetRequestToken'
    authorize_url = 'https://trello.com/1/OAuthAuthorizeToken'
    access_token_url = 'https://trello.com/1/OAuthGetAccessToken'

    expiration = expiration or os.environ.get('TRELLO_EXPIRATION', "30days")
    scope = scope or os.environ.get('TRELLO_SCOPE', 'read,write')
    trello_key = key or os.environ['TRELLO_API_KEY']
    trello_secret = secret or os.environ['TRELLO_API_SECRET']
    name = name or os.environ.get('TRELLO_NAME', 'py-trello')

    # Step 1: Get a request token. This is a temporary token that is used for
    # having the user authorize an access token and to sign the request to obtain
    # said access token.

    session = OAuth1Session(client_key=trello_key, client_secret=trello_secret)
    response = session.fetch_request_token(request_token_url)
    resource_owner_key, resource_owner_secret = response.get('oauth_token'), response.get('oauth_token_secret')

    #if output:
    #    print("Request Token:")
    #    print("    - oauth_token        = %s" % resource_owner_key)
    #    print("    - oauth_token_secret = %s" % resource_owner_secret)
    #    print("")

    # Step 2: Redirect to the provider. Since this is a CLI script we do not
    # redirect. In a web application you would redirect the user to the URL
    # below.

    print("After logging into Trello, go to the following link in your browser and click \"Allow\". ")
    print("There you will be given a verification code.\n")
    print("{authorize_url}?oauth_token={oauth_token}&scope={scope}&expiration={expiration}&name={name}".format(
        authorize_url=authorize_url,
        oauth_token=resource_owner_key,
        expiration=expiration,
        scope=scope,
        name=name
    ))

    # After the user has granted access to you, the consumer, the provider will
    # redirect you to whatever URL you have told them to redirect to. You can
    # usually define this in the oauth_callback argument as well.

    # Python 3 compatibility (raw_input was renamed to input)
    try:
        inputFunc = raw_input
    except NameError:
        inputFunc = input

    oauth_verifier = inputFunc('\nWhat is the verification code? ')

    # Step 3: Once the consumer has redirected the user back to the oauth_callback
    # URL you can request the access token the user has approved. You use the
    # request token to sign this request. After this is done you throw away the
    # request token and use the access token returned. You should store this
    # access token somewhere safe, like a database, for future use.
    session = OAuth1Session(client_key=trello_key, client_secret=trello_secret,
                            resource_owner_key=resource_owner_key, resource_owner_secret=resource_owner_secret,
                            verifier=oauth_verifier)
    access_token = session.fetch_access_token(access_token_url)

    if output:
        #print("Access Token:")
        #print("    - oauth_token        = %s" % access_token['oauth_token'])
        #print("    - oauth_token_secret = %s" % access_token['oauth_token_secret'])
        #print("")
        print("\nYou are all set up.")
        
        credetials = open("credentials.txt", "w")
        credetials.write("%s,%s" %(access_token['oauth_token'],access_token['oauth_token_secret']))
        credetials.close()

    return access_token

if __name__ == '__main__':
    create_oauth_token()

# vim: tabstop=8 expandtab shiftwidth=4 softtabstop=4


## Run Me

The title says it all. But you may wish to edit some variable values to suit your needs. For convenience, I've placed these near the top.

In [ ]:
from trello import TrelloClient
import os, os.path
import re
import json
import numpy as np
from datetime import datetime, timezone

# Load credentials 
keys = open('keys.txt','r').read()
keys = keys.split(",")
credetials = open('credentials.txt','r').read()
credetials = credetials.split(",")
# create client
client = TrelloClient(
    #api_key='your-key', api_secret='your-secret', token='your-oauth-token-key', token_secret='your-oauth-token-secret'
    api_key=keys[0], api_secret=keys[1], token=credetials[0], token_secret=credetials[1]
)

In [ ]:
costperswitch = 0.08
efficency = 0.80
nofBuiltIns = 1

f2vararray = ""
f2divlist = ""

days = 5 # days in which you plan to finish all to do list items
workday = 7 # hours in a work day

#lastMeeting = datetime.strptime('2016-07-25 10:00', '%Y-%m-%d %H:%M')
#nextMeeting = datetime.strptime('2016-08-01 10:00', '%Y-%m-%d %H:%M')
#days = np.busday_count(lastMeeting, nextMeeting, holidays=['2016-07-04', '2016-12-25'])

hoursinweek = days*7*efficency

j = 0

import glob
no_old = len(glob.glob(os.path.join("./summaries/", '*')))

this_sum = no_old + 1
f=open('./summaries/summary.%s.html'%this_sum, 'w+')
f2=open('./clock.html', 'w+')

print("""<!doctype html>
<html lang="en">
<head>
	<meta charset="UTF-8">
	<title>Stopwatch</title>
	<style>
		.timers {
			width:450px;
			margin:0 auto 100px auto;
		}
		.task {
			border-top: 1px solid #aaa;
			padding: 4px 0 0 0;
			margin: 0 0 15px;
		}
		.time {
			margin: 0 0 5px;
		}
		.buttons {
			text-align:right;
		}
	</style>
	<script>
		//	Simple example of using private variables
		//
		//	To start the stopwatch:
		//		obj.start();
		//
		//	To get the duration in milliseconds without pausing / resuming:
		//		var	x = obj.time();
		//
		//	To pause the stopwatch:
		//		var	x = obj.stop();	// Result is duration in milliseconds
		//
		//	To resume a paused stopwatch
		//		var	x = obj.start();	// Result is duration in milliseconds
		//
		//	To reset a paused stopwatch
		//		obj.stop();
		//
		var	clsStopwatch = function() {
				// Private vars
				var	startAt	= 0;	// Time of last start / resume. (0 if not running)
				var	lapTime	= 0;	// Time on the clock when last stopped in milliseconds

				var	now	= function() {
						return (new Date()).getTime(); 
					}; 
		 
				// Public methods
				// Start or resume
				this.start = function() {
						startAt	= startAt ? startAt : now();
					};

				// Stop or pause
				this.stop = function() {
						// If running, update elapsed time otherwise keep it
						lapTime	= startAt ? lapTime + now() - startAt : lapTime;
						startAt	= 0; // Paused
					};

				// Reset
				this.reset = function() {
						lapTime = startAt = 0;
					};

				// Duration
				this.time = function() {
						return lapTime + (startAt ? now() - startAt : 0); 
					};
			};
""", file=f2)

f2posthead = """
		var workday = """+str(workday)+""" 
		var timeondeck = 0
		var totalhours_g = 0

		var nTasks = x.length;
		
		var tq = []
		
		var clocktimer;

		function pad(num, size) {
			var s = "0000" + num;
			return s.substr(s.length - size);
		}

		function formatTime(time) {
			var h = m = s = ms = 0;
			var newTime = '';

			h = Math.floor( time / (60 * 60 * 1000) );
			time = time % (60 * 60 * 1000);
			m = Math.floor( time / (60 * 1000) );
			time = time % (60 * 1000);
			s = Math.floor( time / 1000 );
			ms = time % 1000;

			newTime = pad(h, 2) + ':' + pad(m, 2) + ':' + pad(s, 2) + ':' + pad(ms, 3);
			return newTime;
		}
        
//show OR hide funtion depends on if element is shown or hidden

function shoh(id) { 
    if (document.getElementById) { // DOM3 = IE5, NS6
        if (document.getElementById(id).style.display == "none"){
            document.getElementById(id).style.display = 'block';
        } else {
            document.getElementById(id).style.display = 'none';         
        }   
    } else { 
        if (document.layers) {  
            if (document.id.display == "none"){
                document.id.display = 'block';
            } else {
                document.id.display = 'none';
            }
        } else {
            if (document.all.id.style.visibility == "none"){
                document.all.id.style.display = 'block';
            } else {
                document.all.id.style.display = 'none';
            }
        }
    }
} 

		function show() {		
			for (var i = 0; i < nTasks; i++) {
				tq[i] = document.getElementById(i);
			}
			update();
		}

		function update() {
			var totalhours = 0
			for (var i = 0; i < nTasks; i++) {
				tq[i].innerHTML = formatTime(x[i].time());
				var hid = "h"+i;
				document.getElementById(hid).value = x[i].time();
				totalhours = totalhours + x[i].time();
				
			}
			totalhours_g = totalhours;
			totalhours = totalhours - timeondeck;
			var hoursleft = ((workday*60*60*1000 - totalhours)/(60*60*1000));
			hleft.innerHTML = hoursleft.toFixed(2)
			var d = new Date();
			var newDateObj = new Date(d.getTime() + hoursleft*60*60*1000);
			var ampm = "AM";
			if (newDateObj.getHours() > 12) { 
				ampm = "PM";
				leaveH = newDateObj.getHours() - 12;
			} else {
				leaveH = newDateObj.getHours() - 12;
			}
			if (newDateObj.getMinutes() < 10) { 
				leaveM = "0"+newDateObj.getMinutes();
			} else {
				leaveM = newDateObj.getMinutes();
			}
			var leaveat = leaveH +":"+ leaveM +" "+ ampm
			Closing.innerHTML = leaveat;
		}

		function start(target) {
			clocktimer = setInterval("update()", 1);
			for (var i = 0; i < nTasks; i++) {
				x[i].stop();
			}
			x[target].start();
		}

		function stop(target) {
			for (var i = 0; i < nTasks; i++) {
				x[i].stop();
			}
			clearInterval(clocktimer);
		}

		function reset(target) {
			stop(target);
			x[target].reset();
			update();
		}
	</script>
</head>
<body onload="show();">
<div class=timers>
	<h2><span id='hleft'></span> hours left. Day ends at <span id='Closing'></span></h2>
	<div class=task>
		<div class=time>Other</div>
		<div class=buttons>
			(<span id='0'></span>) <input type=hidden id='h0'/>
			<input type="button" value="start" onclick="start(0);">
			<input type="button" value="stop" onclick="stop(0);">
			<input type="button" value="reset" onclick="reset(0)">
		</div>
	</div>
"""

print ("""<HTML><HEAD><STYLE>BODY {\n\tfont-family: Calibri, Candara, Segoe, Optima, Arial, sans-serif;\n\tfont-size: 11pt;\n}</STYLE></HEAD>
<BODY>""", file=f)
print ("Text <ins style='background:#ddffdd;'>highlighted in green</ins> represents additions, and text <del style='background:#ffdddd;'>highlighted in red</del> represents deletions.<br>", file=f)

boards = client.list_boards()
#print (boards)
for b in boards:
    #print (dir(b))
    # Exclude Boards as you wish. For example, I have personal and work board, and never should the two meet.
    if "Groceries" not in b.name and "Suffolk" not in b.name and "Personal" not in b.name and False == b.closed:
        lists = b.all_lists()
        i = 0
        for c in lists:
            #print (dir(c))
            
            if "On Deck" in c.name or "Waiting" in c.name or "Doing" in c.name or "Done" in c.name:
                cards = c.list_cards()
                
                # If list contains "Doing", add board name to time tracking doc.
                if len(cards) > 0 and "Doing" in c.name:
                    f2divlist = f2divlist+"""
	<h2>"""+b.name+"""</h2>\n"""
                
                # First time through? Add Board as heading to summary doc.
                if i == 0:
                    print ("<br>===============================<br>\n", file=f)
                    print ("&nbsp;&nbsp;<b>%s</b><br>\n"%b.name, file=f)
                    print ("===============================<br>\n", file=f)
                
                # If there are cards, add them to summary doc.
                i = i+1
                if len(cards) > 0:
                    print ("<br><b>%s</b><br>\n"%c.name, file=f)
                    # Loop through cards
                    for d in cards:
                        task = re.sub(r"(http(|s):[^\s]*)", r"<a href='\1'>\1</a>", d.name)
                        if "Doing" in c.name:
                            hours = re.match( r'^\[(\d+\.?\d*)\]', task)
                            if hours:
                                if float(hours.group(1)) > 0:
                                    hoursinweek = hoursinweek - float(hours.group(1)) - costperswitch
                                #print (d.name)
                                #print ("hours: %s"%hours.group(1))
                                #print (hoursinweek)
                                if j == 0:
                                    f2vararray = "		var x = [new clsStopwatch()"
                                    k=0
                                    while k<nofBuiltIns:
                                        f2vararray = f2vararray+",new clsStopwatch()"
                                        k=k+1
                                    j=nofBuiltIns
                                else: 
                                    f2vararray = f2vararray+",new clsStopwatch()"
                                
                                f2divlist = f2divlist+"	<div class=task id='h"+str(j)+"' style='display:none;'><font size=-1><a href='javascript:shoh(\"d"+str(j)+"\");shoh(\"h"+str(j)+"\")' style=\"color:gray; text-decoration:none;\">"+d.name+"</a></font></div>"
                                f2divlist = f2divlist+"	<div class=task id='d"+str(j)+"'>\n		<div class=time><a href='javascript:shoh(\"d"+str(j)+"\");shoh(\"h"+str(j)+"\")'  style=\"color:black; text-decoration:none;\">"+d.name+"</a></div>"
                                f2divlist = f2divlist+"""
		<div class=buttons>
			(<span id='"""+str(j)+"""'></span>) <input type=hidden id='h"""+str(j)+"""'/>
			<input type="button" value="start" onclick="start("""+str(j)+""");">
			<input type="button" value="stop" onclick="stop("""+str(j)+""");">
			<input type="button" value="reset" onclick="reset("""+str(j)+""")">
		</div>
	</div>\n"""
                                
                                
                                j = j + 1
                                
                        task = re.sub(r"(^(\[\d+\.?\d*\])|(\[\d+\.?\d*\]\s*)$)", r"", task)
                                    
                        print ("+ %s<br>\n"%(task), file=f)
                        comments = d.get_comments()
                        for e in comments:
                            comment = re.sub(r"(http(|s):[^\s]*)", r"<a href='\1'>\1</a>", e['data']['text'])
                            comment = re.sub(r"\n\-\s{1}", "\n", comment)
                            comment = re.sub(r"^\-\s{1}", "", comment)
                            comment = re.sub(r"\n", "<br>\n---- ", comment)
                            comment = re.sub(r"\n---- <br>", "", comment)
                            print ("-- %s<br>\n"%comment, file=f)
                        print ("\n", file=f)

print ("</BODY></HTML>", file=f)
f.close()

f2vararray = f2vararray+"];\n"
print (f2vararray, file=f2)
print (f2posthead, file=f2)
print (f2divlist, file=f2)

print("	<hr>	<input type=input id=oneday value=\""+str(workday)+"\" maxlength=\"4\"><input type=button onClick=\"workday=document.getElementById('oneday').value;timeondeck=totalhours_g;update();\" value=\"New Day\">\n</div>\n</body>\n</html>", file=f2)
f2.close()

print("\nHours available in the upcoming week minus hours estimated in your To Do lists: %.2f"%hoursinweek)

In [ ]:
#h/t http://stackoverflow.com/questions/774316/python-difflib-highlighting-differences-inline
    
import difflib
def show_diff(seqm):
    """Unify operations between two compared strings seqm is a difflib.SequenceMatcher instance whose a & b are strings"""
    output= []
    for opcode, a0, a1, b0, b1 in seqm.get_opcodes():
        if opcode == 'equal':
            output.append(seqm.a[a0:a1])
        elif opcode == 'insert':
            output.append("<ins style=\"background:#ddFFdd\">" + seqm.b[b0:b1] + "</ins>")
        elif opcode == 'delete':
            output.append("<del style=\"background:#FFdddd\">" + seqm.a[a0:a1] + "</del>")
        #elif opcode == 'replace':
        #    raise NotImplementedError, "what to do with 'replace' opcode?"
        #else:
        #    raise RuntimeError, "unexpected opcode"
    return ''.join(output)

if this_sum > 1:
    fromfile = "./summaries/summary.%s.html"%no_old
    tofile = "./summaries/summary.%s.html"%this_sum
    fromlines = open(fromfile, 'r').read()
    tolines = open(tofile, 'r').read()

    sm= difflib.SequenceMatcher(None, fromlines, tolines)

    credetials = open("summary.html", "w")
    credetials.write(show_diff(sm))
    credetials.close()
else:
    tofile = "./summaries/summary.%s.html"%this_sum
    tolines = open(tofile, 'r').read()

    credetials = open("summary.html", "w")
    credetials.write(tolines)
    credetials.close()    
    

## Output

- [Summary document](summary.html)
- [Time tracking document](clock.html) 

## To Do

- Add to time tracking doc so you can save its data (e.g., Project, task, and task time by day) to a json file in this folder. 
- Build some anaylitics in this notebook that allow you to explore the time tracking data stored in saved json files. 
- Order boards based on order in *Stared Board* as opposed to alphabetically. 
- Remove *Waiting* lists in Trello and fill them here by looking for cards where the assigned user isn't you.